In [1]:
from unet import UNet
import torch

unet_model_path = "unet_model.pth"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
unet_model = UNet().to(device)
unet_model.load_state_dict(torch.load(unet_model_path))
unet_model.eval()

/tmp/ipykernel_3839065/4013129702.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  unet_model.load_state_dict(torch.load(unet_model_path))


UNet(
  (enc1): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
  )
  (enc2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
  )
  (enc3): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_r

In [2]:
from resnet import ResNetCNN

resnet_model_path = "resnet_model.pth"

#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet_model = ResNetCNN().to(device)
resnet_model.load_state_dict(torch.load(resnet_model_path))
resnet_model.eval()

/opt/anaconda3/envs/env_lsy/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/envs/env_lsy/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/tmp/ipykernel_3839065/2200365919.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECUR

ResNetCNN(
  (resnet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_ru

In [3]:
from dataset import RESNETTestset
from torch.utils.data import DataLoader

test1_dir = '../chest_xray/test/NORMAL'
test2_dir = '../chest_xray/test/PNEUMONIA'

test1_dataset = RESNETTestset(unet_model, device, test1_dir, target_size=(256, 256))
test1_loader = DataLoader(test1_dataset, batch_size=1)

test2_dataset = RESNETTestset(unet_model, device, test2_dir, target_size=(256, 256))
test2_loader = DataLoader(test2_dataset, batch_size=1)

In [4]:
with torch.no_grad():
    test1_len = 0
    test1_count = 0
    for idx, image in enumerate(test1_loader):
        image = image.to(device)
        output = resnet_model(image.repeat(1, 3, 1, 1))
        prediction = torch.sigmoid(output).cpu()
        eval = prediction[0, 0].item() > prediction[0, 1].item()
        #print(idx, prediction)
        
        test1_len += 1
        if eval:
            test1_count += 1
            
        #if idx > 10: break
            
    test2_len = 0
    test2_count = 0
    for idx, image in enumerate(test2_loader):
        image = image.to(device)
        output = resnet_model(image.repeat(1, 3, 1, 1))
        prediction = torch.sigmoid(output).cpu()
        eval = prediction[0, 0].item() < prediction[0, 1].item()
        #print(idx, prediction)
        
        test2_len += 1
        if eval:
            test2_count += 1
        
        #if idx > 10: break
    
    print(f'test1 : {100 * test1_count / test1_len:.2f}%')
    print(f'test2 : {100 * test2_count / test2_len:.2f}%')

test1 : 50.85%
test2 : 99.74%
